In [5]:
import pandas as pd
import numpy as np
from copy import deepcopy
from pandas import datetime
import gc
import random
import datetime
import csv
from os import listdir
from os.path import isfile, join, isdir
from tqdm import tqdm_notebook
from Preprocessing import Preprocessor, set_onelevel, set_multilevel
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [6]:
# getting file tree in directory "data_csv", which contains raw unfiltered data
mypath = '/home/ndsviriden/data_csv/Raw'
#dateparse = pd.to_datetime
station_files = [f for f in listdir(mypath) if isfile(join(mypath, f)) if int(f.split('_', 1)[0])]
station_files.sort()
station_files

# reading loaded csv files from data_csv directory, output is a list (length=number of stations) of raw df 
raw_lmeteo_list = [pd.read_csv(mypath + '/' + station, index_col=0, 
                    dtype={'station_id': int,'date_time': str}, 
                    parse_dates=True) 
                   for station in tqdm_notebook(station_files[18:20])]

#fix problems with indexing, later this will be solved in loading part
for elem in raw_lmeteo_list:
    elem.reset_index(drop=True, inplace=True)

raw_data = pd.concat(raw_lmeteo_list)
raw_data['id'] = random.sample(range(len(raw_data)), len(raw_data))
raw_data['valid'] = True
raw_data = raw_data.reset_index(drop=True)

del raw_lmeteo_list
gc.collect()

raw_data.head()

,station_data_id,sensor_id,data,station_id,sensor_type_id,active_state,name,rule_id,rule_ordr,direction,lane_traffic,date_time,poll_interval,fake_data,type,unit_of_measure,description,id,valid
0,6466159.0,5410.0,744.0,303,16.0,A,NaN,152.0,36.0,0.0,0.0,2012-07-12 09:14:31,NaN,NaN,pressure,NaN,pressure (sw),188890,True
1,6466159.0,5399.0,252.0,303,2.0,A,NaN,154.0,14.0,0.0,0.0,2012-07-12 09:14:31,NaN,NaN,t_road,NaN,t_road (sw),224429,True
2,6466159.0,5398.0,241.0,303,1.0,A,NaN,4.0,12.0,0.0,0.0,2012-07-12 09:14:31,NaN,NaN,t_air,NaN,t_air (sw),368944,True
3,6466159.0,5400.0,253.0,303,3.0,A,NaN,155.0,16.0,0.0,0.0,2012-07-12 09:14:31,NaN,NaN,t_underroad,NaN,t_underroad (sw),486642,True
4,6466159.0,5401.0,596.0,303,4.0,A,NaN,6.0,18.0,0.0,0.0,2012-07-12 09:14:31,NaN,NaN,dampness,NaN,dampness (sw),378164,True


In [8]:
raw_data.date_time

0         2012-07-12 09:14:31
1         2012-07-12 09:14:31
2         2012-07-12 09:14:31
3         2012-07-12 09:14:31
4         2012-07-12 09:14:31
5         2012-07-12 09:41:08
6         2012-07-12 09:41:08
7         2012-07-12 09:41:08
8         2012-07-12 09:41:08
9         2012-07-12 09:41:08
10        2012-07-12 09:48:47
11        2012-07-12 09:48:47
12        2012-07-12 09:48:47
13        2012-07-12 09:48:47
14        2012-07-12 09:48:47
15        2012-08-31 20:34:38
16        2012-08-31 20:34:38
17        2012-08-31 20:34:38
18        2012-08-31 20:34:38
19        2012-08-31 20:34:38
20        2012-08-31 20:34:38
21        2012-08-31 20:34:38
22        2012-08-31 20:34:38
23        2012-08-31 20:34:38
24        2012-08-31 20:34:38
25        2012-08-31 21:04:50
26        2012-08-31 21:04:50
27        2012-08-31 21:04:50
28        2012-08-31 21:04:50
29        2012-08-31 21:04:50
                 ...         
669580    2016-07-13 16:39:47
669581    2016-07-13 16:39:47
669582    

## Create multicolumn table and write it into .csv

In [3]:
preprocessor = Preprocessor()
meteo_split = preprocessor.PivotTable(raw_data)
meteo_split.to_csv('test_table.csv')

## Parse csv into multicolumn Dataframe

In [4]:
df = pd.read_csv('test_table.csv', header=[0, 1], skipinitialspace=True, tupleize_cols=True, index_col=0)
columns = [(elem[0], '') if elem[0] in ('date_time', 'station_id') else elem for elem in df.columns]
df.columns = pd.MultiIndex.from_tuples(columns)
df[('date_time', '')] = pd.to_datetime(df[('date_time', '')])
df = df.reset_index(drop=True)
df.head()

station_id           date_time     data                                     \
                                 dampness pressure  t_air t_road t_underroad   
0        303 2012-07-12 09:14:31    596.0    744.0  241.0  252.0       253.0   
1        303 2012-07-12 09:41:08    610.0    745.0  240.0  248.0       250.0   
2        303 2012-07-12 09:48:47    598.0    744.0  241.0  250.0       251.0   
3        303 2012-08-31 20:34:38    807.0    751.0  115.0  126.0       127.0   
4        303 2012-08-31 21:04:50    829.0    751.0  111.0  117.0       118.0   

         id                                              valid                 \
   dampness  pressure     t_air    t_road t_underroad dampness pressure t_air   
0   74763.0  547924.0  661700.0  137834.0    489807.0     True     True  True   
1  406823.0   70244.0  219860.0  539262.0     81565.0     True     True  True   
2  619335.0  198573.0  376654.0  285716.0     68513.0     True     True  True   
3  208898.0  420623.5  177017.0  155260.5    277976.0     True     True  True   
4  309165.5  557108.5  350179.0  326280.0    440425.0     True     True  True   

                      
  t_road t_underroad  
0   True        True  
1   True        True  
2   True        True  
3   True        True  
4   True        True